## Dataset Split

In [16]:
import os 
import datetime
from pathlib import Path
from dotenv import load_dotenv, find_dotenv
import pandas as pd
import re

basepath = Path(os.getcwd())
# make sure your working directory is the repository root.
if basepath.name != "idp-radio-1":
    os.chdir(basepath.parent.parent)
load_dotenv(find_dotenv())

%load_ext autoreload
%autoreload 2
os.getcwd()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'/srv/idp-radio-1'

In [4]:
#!wget http://download.cs.stanford.edu/deep/CheXpert-v1.0-small.zip


In [5]:
load_dotenv(find_dotenv())
chestxray14_folder = Path(os.environ.get("CHESTXRAY14_DATASET_DIRECTORY"))
chexpert_folder = Path(os.environ.get("CHEXPERT_DATASET_DIRECTORY"))
print(chestxray14_folder)
print(chexpert_folder)

data/chestxray14/scale_256
data/chexpert/full


In [6]:
chestxray14_labels = pd.read_csv(chestxray14_folder / "meta/data/labels.csv")
chexpert_train_labels = pd.read_csv(chexpert_folder / "train.csv")
chexpert_test_labels = pd.read_csv(chexpert_folder / "valid.csv")

In [7]:
chestxray14_labels.head()

,Image Index,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage_Width,OriginalImage_Height,OriginalImagePixelSpacing_x,OriginalImagePixelSpacing_y,...,Infiltration,Consolidation,Fibrosis,No Finding,Effusion,Nodule,Mass,Hernia,Emphysema,Pneumothorax
0,00000001_000.png,0,1,57,M,PA,2682,2749,0.143,0.143,...,False,False,False,False,False,False,False,False,False,False
1,00000001_001.png,1,1,58,M,PA,2894,2729,0.143,0.143,...,False,False,False,False,False,False,False,False,True,False
2,00000001_002.png,2,1,58,M,PA,2500,2048,0.168,0.168,...,False,False,False,False,True,False,False,False,False,False
3,00000002_000.png,0,2,80,M,PA,2500,2048,0.171,0.171,...,False,False,False,True,False,False,False,False,False,False
4,00000003_001.png,0,3,74,F,PA,2500,2048,0.168,0.168,...,False,False,False,False,False,False,False,True,False,False


In [8]:
chexpert_train_labels.head()

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,CheXpert-v1.0-small/train/patient00001/study1/...,Female,68,Frontal,AP,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0
1,CheXpert-v1.0-small/train/patient00002/study2/...,Female,87,Frontal,AP,NaN,NaN,-1.0,1.0,NaN,-1.0,-1.0,NaN,-1.0,NaN,-1.0,NaN,1.0,NaN
2,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Frontal,AP,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
3,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Lateral,NaN,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
4,CheXpert-v1.0-small/train/patient00003/study1/...,Male,41,Frontal,AP,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [9]:
chexpert_test_labels.head()

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,CheXpert-v1.0-small/valid/patient64541/study1/...,Male,73,Frontal,AP,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,CheXpert-v1.0-small/valid/patient64542/study1/...,Male,70,Frontal,PA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,CheXpert-v1.0-small/valid/patient64542/study1/...,Male,70,Lateral,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,CheXpert-v1.0-small/valid/patient64543/study1/...,Male,85,Frontal,AP,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,CheXpert-v1.0-small/valid/patient64544/study1/...,Female,42,Frontal,AP,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
!ls data/chexpert/full/CheXpert-v1.0-small/train/patient28746/

study1	 study19  study28  study37  study46  study55  study64  study73	study82
study10  study2   study29  study38  study47  study56  study65  study74	study83
study11  study20  study3   study39  study48  study57  study66  study75	study84
study12  study21  study30  study4   study49  study58  study67  study76	study85
study13  study22  study31  study40  study5   study59  study68  study77	study86
study14  study23  study32  study41  study50  study6   study69  study78	study9
study15  study24  study33  study42  study51  study60  study7   study79
study16  study25  study34  study43  study52  study61  study70  study8
study17  study26  study35  study44  study53  study62  study71  study80
study18  study27  study36  study45  study54  study63  study72  study81


In [28]:
def get_patient_id (row):
    patient_folder = re.findall(r"patient[0-9]+", row["Path"])
    if len(patient_folder) > 0:
        patient_number = re.findall(r"[0-9]+", patient_folder[0])
    if len(patient_number) > 0:
        return (patient_number[0])

In [29]:
chexpert_train_labels["patient_id"] = chexpert_train_labels.apply(get_patient_id, axis = 1)
chexpert_test_labels["patient_id"] = chexpert_test_labels.apply(get_patient_id, axis = 1)


In [23]:
chexpert_test_labels.head()

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,patient_id
0,CheXpert-v1.0-small/train/patient00001/study1/...,Female,68,Frontal,AP,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0,1
1,CheXpert-v1.0-small/train/patient00002/study2/...,Female,87,Frontal,AP,NaN,NaN,-1.0,1.0,NaN,-1.0,-1.0,NaN,-1.0,NaN,-1.0,NaN,1.0,NaN,2
2,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Frontal,AP,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2
3,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Lateral,NaN,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2
4,CheXpert-v1.0-small/train/patient00003/study1/...,Male,41,Frontal,AP,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,3


In [33]:
chexpert_train_labels["patient_id"].value_counts()

28746    92
33155    92
04462    89
24163    86
34615    85
         ..
58723     1
05100     1
41507     1
63492     1
07717     1
Name: patient_id, Length: 64540, dtype: int64

In [34]:
chexpert_test_labels["patient_id"].value_counts()

64616    3
64547    3
64581    3
64595    2
64560    2
        ..
64694    1
64661    1
64692    1
64730    1
64669    1
Name: patient_id, Length: 200, dtype: int64

In [88]:
#!mkdir ~/idp-radio-1/data/dev_dataset
#!mkdir ~/idp-radio-1/data/dev_dataset/CheXpert-v1.0-small
#!mkdir ~/idp-radio-1/data/dev_dataset/CheXpert-v1.0-small/train
!ls ~/idp-radio-1/data/dev_dataset/CheXpert-v1.0-small/train

In [70]:
chexpert_train_labels.to_csv(chexpert_folder / "train.csv")
chexpert_test_labels.to_csv(chexpert_folder / "test.csv")